# students-to-course-assigner

## Imports

In [ ]:
import os

import pandas as pd
import math

from dataclasses import dataclass, field
from typing import List

import random

from collections import defaultdict

## Read data

In [ ]:
excel_path = os.path.join("data", "InputNeufeld_Dez2023.xlsx")

In [ ]:
df_students = pd.read_excel(excel_path, sheet_name="Rohdaten").fillna("").reset_index()
df_courses = pd.read_excel(excel_path, sheet_name="Module")

In [ ]:
df_students.head()

In [ ]:
df_courses.head().fillna("")

In [ ]:
@dataclass
class Student:
    id_: int
    first_name: str
    last_name: str
    desired_courses: List[str]
    assigned_course: str = None

In [ ]:
s = Student(1, "j", "l", ["a1", "a2", "a3"])

In [ ]:
s

In [ ]:
student_per_student_id = {}
for e in df_students.to_dict(orient="records"):
    cs = []
    for i in range(1, 5):
        c = e[f"Def B - {i}. Prio"].replace(" ", "")
        cs += [c]
    if not all(not c for c in cs):
        if any(not c for c in cs):
            print(f"student has not all courses not defined, but any: {e}")
        else:
            id_ = e["index"]
            student_per_student_id[id_] = Student(id_, e["Vorname"], e["Name"], cs)
    else:
        print(f"student has all courses not defined: {e}")

In [ ]:
print(len(student_per_student_id))

In [ ]:
@dataclass
class Course:
    id_: str
    min_students: int
    max_students: int
    nb_wishes: int = 0
    assigned_students: List[int] = field(default_factory=lambda : [])
    def assign_if_possible(self, student):
        if len(self.assigned_students) < self.max_students:
            self.assigned_students += [student]
            student.assigned_course = self

In [ ]:
course_per_course_id = {}
for e in df_courses.to_dict(orient="records"):
    id_ = e["Module"]
    if id_.startswith("B"):
        course_per_course_id[id_] = Course(id_, e["Min"], e["Max"])

In [ ]:
print(len(course_per_course_id))

In [ ]:
nb_wishes_per_course_id = defaultdict(int)
for s in student_per_student_id.values():
    for c_id in s.desired_courses:
        nb_wishes_per_course_id[c_id] += 1
for c in course_per_course_id.values():
    c.nb_wishes = nb_wishes_per_course_id[c.id_]

In [ ]:
courses_to_remove = []
for c in course_per_course_id.values():
    if c.nb_wishes < c.min_students:
        courses_to_remove += [c]
for c in courses_to_remove:
    print(f"course {c} has not enough wishes, is removed")
    del course_per_course_id[c.id_]

## Heuristic approach

In [ ]:
def init_assignment():
    for s in student_per_student_id.values():
        s.assigned_course = None
    for c in course_per_course_id.values():
        c.assigned_students = []

In [ ]:
def shuffle_list(ls):
    random.shuffle(ls)
    return ls

In [ ]:
init_assignment()
for s in shuffle_list(list(student_per_student_id.values())):
    n = len(s.desired_courses)
    a_desired_course = s.desired_courses[random.randint(0, n-1)]
    course = course_per_course_id[a_desired_course]
    course.assign_if_possible(s)

In [ ]:
students_without_course = [s for s in student_per_student_id.values() if s.assigned_course is None]
print(len(students_without_course))

In [ ]:
for c in course_per_course_id.values():
    print(c.id_, c.min_students, c.max_students, c.nb_wishes / 4, len(c.assigned_students), c.nb_wishes < c.min_students)

## With integer programming

In [ ]:
from ortools.sat.python import cp_model

In [ ]:
model = cp_model.CpModel()

In [ ]:
x = {}
for student_ind, _ in enumerate(student_per_student_id.values()):
    for course_ind, _ in enumerate(course_per_course_id.values()):
        x[student_ind, course_ind] = model.NewBoolVar(f"x[{student_ind},{course_ind}]")

In [ ]:
# Each student is assigned to exactly one course.
for student_ind, _ in enumerate(student_per_student_id.values()):
    model.AddExactlyOne(x[student_ind, course_ind] for (course_ind, course) in enumerate(course_per_course_id.values()))

In [ ]:
# Each course takes at most n students
for course_ind, course in enumerate(course_per_course_id.values()):
    model.Add(sum([x[student_ind, course_ind] for (student_ind, _) in enumerate(student_per_student_id.values())]) >= course.min_students)
    model.Add(sum([x[student_ind, course_ind] for (student_ind, _) in enumerate(student_per_student_id.values())]) <= course.max_students)

In [ ]:
def costs(student, course_id):
    if course_id in student.desired_courses:
        return student.desired_courses.index(course_id)
    else:
        return 1000

In [ ]:
a_student = list(student_per_student_id.values())[10]
print(a_student)

In [ ]:
costs(a_student, "B21")

In [ ]:
objective_terms = []
for student_ind, student in enumerate(student_per_student_id.values()):
    for course_ind, course in enumerate(course_per_course_id.values()):
        objective_terms += [costs(student, course.id_) * x[student_ind, course_ind]]
model.Minimize(sum(objective_terms))

In [ ]:
solver = cp_model.CpSolver()

In [ ]:
status = solver.Solve(model)

In [ ]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"Total cost = {solver.ObjectiveValue()}\n")
    for student_ind, student in enumerate(student_per_student_id.values()):
        for course_ind, course in enumerate(course_per_course_id.values()):
            if solver.BooleanValue(x[student_ind, course_ind]):
                student.assigned_course = course.id_
                course.assigned_students += [student]
else:
    print("No solution found.")

In [ ]:
students_without_course = [s for s in student_per_student_id.values() if s.assigned_course is None]
print(len(students_without_course))

In [ ]:
students_with_not_desired_course = [s for s in student_per_student_id.values() if not s.assigned_course in s.desired_courses]
print(len(students_with_not_desired_course))

In [ ]:
for course in course_per_course_id.values():
    nb_students = len(course.assigned_students)
    if nb_students < course.min_students:
        print(f"not enough students: {course, nb_students}")
    if nb_students > course.max_students:
        print(f"too much students: {course, nb_students}")

In [ ]:
for c in course_per_course_id.values():
    print(c.id_, c.min_students, c.max_students, c.nb_wishes, len(c.assigned_students))